In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
sns.set()
from scipy.stats import skew

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data2015=pd.read_csv('2015-building-energy-benchmarking.csv') #3340 rows × 47 columns
data2016=pd.read_csv('2016-building-energy-benchmarking.csv') #3376 rows × 46 columns

# Concaténation 2015/2016; uniquement colonnes intéressantes

In [ ]:
data2015.columns

In [ ]:
data2016.columns

In [ ]:
cols=[ 'BuildingType','PrimaryPropertyType', 'Neighborhood', 'YearBuilt', 
      'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking','PropertyGFABuilding(s)',
      'YearsENERGYSTARCertified', 'ENERGYSTARScore','SiteEnergyUse(kBtu)']

In [ ]:
GHGE_2015=['GHGEmissions(MetricTonsCO2e)']
GHGE_2016=['TotalGHGEmissions']

In [ ]:
short_2015=data2015.loc[:,cols+GHGE_2015]
short_2016=data2016.loc[:,cols+GHGE_2016]

In [ ]:
short_2016.rename(columns={"TotalGHGEmissions":"GHGEmissions(MetricTonsCO2e)"},inplace=True)

In [ ]:
df=pd.concat([short_2015,short_2016])

In [ ]:
df

# Uniquement les bâtiments non résidentiels 

In [ ]:
df['BuildingType'].unique()

In [ ]:
non_residential_columns=['NonResidential', 'Nonresidential COS', 'Nonresidential WA', 'SPS-District K-12', 'Campus' ]

In [ ]:
NonRes=df[df['BuildingType'].isin(non_residential_columns)]

In [ ]:
NonRes

In [ ]:
NonRes['BuildingType'].unique()

# Nettoyage 

## Supression des lignes dont les variables cibles sont manquantes

In [ ]:
NonRes.drop(NonRes[NonRes['SiteEnergyUse(kBtu)'].isna()].index, inplace=True)

NonRes.drop(NonRes[NonRes['GHGEmissions(MetricTonsCO2e)'].isna()].index, inplace=True)

## Visualisation 

In [ ]:
NonRes.describe()

In [ ]:
numeric_cols=NonRes.describe().columns

In [ ]:
fig = plt.figure(figsize = (15, 20))

for i in range(len(numeric_cols)):
    graphique=plt.subplot(6,3,i+1)
    NonRes.boxplot(column=numeric_cols[i])
    graphique.set_xticks([])
    graphique.set_title(numeric_cols[i])

fig.tight_layout()

## Traitement des erreurs lexicales

In [ ]:
for i in numeric_cols:
    pd.to_numeric(NonRes[i] , errors='coerce')

In [ ]:
def neigborhood_categ(x):
    if x in ['Delridge','DELRIDGE NEIGHBORHOODS']:
        return('DELRIDGE')
    elif x==np.nan:
        pass
    else:
        return(x.upper())

In [ ]:
NonRes['Neighborhood']=NonRes['Neighborhood'].apply(neigborhood_categ)

## Traitement des valeurs aberrantes

In [ ]:
def intervalle_NaN(x,mini,maxi):# x = Series 
    if x==np.nan:
        pass
    elif (x<mini or x>maxi):
        return(np.nan)
    else :
        return(x)

In [ ]:
NonRes['YearBuilt']=NonRes['YearBuilt'].apply(intervalle_NaN, args=[1900,2016])#rénovation possible jusqu'en 2016
NonRes['NumberofFloors']=NonRes['NumberofFloors'].apply(intervalle_NaN, args=[0,100])#1 super tall 101 story-Building in Seattle
NonRes['PropertyGFATotal']=NonRes['PropertyGFATotal'].apply(intervalle_NaN, args=[0,3000000])#3*10^6, pour supprimer un outlier
NonRes['PropertyGFAParking']=NonRes['PropertyGFAParking'].apply(intervalle_NaN, args=[0,max(NonRes['PropertyGFAParking'])])# que ce ne soit pas <0
NonRes['PropertyGFABuilding(s)']=NonRes['PropertyGFABuilding(s)'].apply(intervalle_NaN, args=[0,3000000])#3*10^6, pour supprimer un outlier
NonRes['ENERGYSTARScore']=NonRes['ENERGYSTARScore'].apply(intervalle_NaN, args=[0,100])# Par définition 
NonRes['SiteEnergyUse(kBtu)']=NonRes['SiteEnergyUse(kBtu)'].apply(intervalle_NaN, args=[0,300000000])#3*10^8, pour supprimer un outlier
NonRes['GHGEmissions(MetricTonsCO2e)']=NonRes['GHGEmissions(MetricTonsCO2e)'].apply(intervalle_NaN, args=[0,12500])#outlier

##### On ne garde que les GHGE positifs car on ne prend pas en compte d'éventuels bâtiments suffisamment performants pour absorber le CO2 d'autre source de pollution. En effet, présents en trop petit nombre, il faudrait faire un modèle spécifique pour eux : pas très rentables ...

##### On a introduit de nouveaux NaN dans les variables cibles, il faut les supprimer:

In [ ]:
NonRes.drop(NonRes[NonRes['SiteEnergyUse(kBtu)'].isna()].index, inplace=True)
NonRes.drop(NonRes[NonRes['GHGEmissions(MetricTonsCO2e)'].isna()].index, inplace=True)

##### la production totale doit être (au moins superieure) à la somme des productions des bâtiments + des parkings :

In [ ]:
for i in range(len(NonRes)):
      if NonRes['PropertyGFATotal'].iloc[i]<(NonRes['PropertyGFAParking'].iloc[i]+NonRes['PropertyGFABuilding(s)'].iloc[i]):
        NonRes['PropertyGFATotal'].iloc[i]=np.nan
        NonRes['PropertyGFAParking'].iloc[i]=np.nan
        NonRes['PropertyGFABuilding(s)'].iloc[i]=np.nan

## Vérification 

In [ ]:
NonRes.describe()

In [ ]:
fig = plt.figure(figsize = (20, 30), )

for i in range(len(numeric_cols)):
    graphique=plt.subplot(6,4,i+1)
    NonRes.boxplot(column=numeric_cols[i], fontsize=15)
    graphique.set_xticks([])
    graphique.set_title(numeric_cols[i],fontsize=20 )

fig.tight_layout()

## Taux de remplissage des colonnes 

In [ ]:
T=round(NonRes.count().sum()/(len(NonRes)*len(NonRes.columns))*100,2)
print('le taux de remplissage total du dataframe est de :', T,'%' )

In [ ]:
Taux=pd.DataFrame([{'colonne':cols, 
                    'taux': round(NonRes[cols].count()/len(NonRes),2), 
                    'taux_missing':round(1-NonRes[cols].count()/len(NonRes),2)} for cols in NonRes])
Taux

In [ ]:
fig,ax=plt.subplots(figsize=(20,8))

present = Taux['taux']
missing = Taux['taux_missing']
ind = np.arange(len(Taux))    # the x locations for the groups

p1 = plt.bar(ind, present,color=(0.2, 0.4, 0.6, 0.6))
p2 = plt.bar(ind, missing, bottom=present, color=(0.92, 0.73, 0.5,0.4))

plt.ylabel('Taux', fontsize=20)
plt.yticks(np.arange(0, 1, 0.2), fontsize=15)

ax.set_xticks(np.arange(len(Taux)))
ax.set_xticklabels(Taux['colonne'], fontsize=20)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

plt.title('Taux de remplissage des variables',  fontsize=25)

_=plt.legend((p1[0], p2[0]), ('present values', 'missing values'),fontsize=25)

##### La colonne YearsENERGYSTARCertified est trop peu remplie et n'apporte pas tellement d'information pertinante: on la supprime. 
##### Toutes les autres colonnes sont très bien remplies : on a un df de bonne qualité. 
##### Seule ENERGYSTARscore laisse encore à désirer mais on cherche justement à déterminer l'utilité de cette colonne dans ce projet... 

In [ ]:
NonRes.drop(columns='YearsENERGYSTARCertified', inplace=True)

In [ ]:
T=round(NonRes.count().sum()/(len(NonRes)*len(NonRes.columns))*100,2)
print('le taux de remplissage total du dataframe est de :', T,'%' )

## Supression des éventuelles lignes qui ne seraient pas assez remplies 

  ### Taux de remplissge des lignes

In [ ]:
df_copy=NonRes.notna()
NonRes['taux']=df_copy.apply(sum, axis=1)
NonRes['taux']=NonRes['taux']/(len(df_copy.columns))#len(df_copy) pour ne pas comptabiliser taux

### nbr de lignes restantes en fonction du taux de remplissage 

In [ ]:
nbr_ligne=[]
taux_ligne=[]
nbr_manque=[]

for i in range (0,11):
    C=NonRes['taux'][NonRes['taux']>=i*0.1].count()
    taux_ligne.append(i*0.1)
    nbr_manque.append(round((len(NonRes.columns)-1)*(1-(i*0.1))))# -1 pour ne pas comptabiliser taux 
    nbr_ligne.append(int(C))
    
name=pd.DataFrame({'taux_remplissage_mini':taux_ligne,'nbr_colonne_vide':nbr_manque, 'nbr_lignes_restantes':nbr_ligne})
name

### Transformation du df

In [ ]:
NonRes=NonRes[NonRes['taux']>= 0.9] # on choisi un taux de remplissage de 90% minimum
NonRes.drop(columns=['taux'], inplace=True)
NonRes.reset_index(drop=True,inplace=True)

In [ ]:
T=round(NonRes.count().sum()/(len(NonRes)*len(NonRes.columns))*100,2)
print('le taux de remplissage total du dataframe est de :', T,'%' )

## Imputation des valeurs manquantes 

In [ ]:
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

In [ ]:
# on n'impute PAS de valeurs aux variables cibles : 2 dernieres colonnes 
# ni aux 3 permieres cols puisque ce sont des string
# Ni à ENERGYSTARScore puisqu'on cherche à estimer son utilité
imp = IterativeImputer(skip_complete=True, random_state=0,
                       min_value=[1900, 0, 0, 0, 0], 
                       max_value=[2016, 100, 3000000, 3000000, 3000000])
NonRes[NonRes.iloc[:,3:8].columns] = imp.fit_transform(NonRes.iloc[:,3:8])
#NonRes

## Un petit ajout (juste pour la visualisation)...

In [ ]:
def Principal_Use(x):
    if x in ['Small- and Mid-Sized Office','Office']:
        return('Small- and Mid-Sized Office')
    
    elif x in ['Distribution Center','Distribution Center\n']:
        return('Distribution Center')
    
    elif x in ['Restaurant','Restaurant\n']:
        return('Restaurant')
    
    elif x in ['Supermarket / Grocery Store', 'Supermarket/Grocery Store',]:
        return('Grocery Store')
    
    elif x in ['K-12 School','SPS-District K-12','University','College/University']:
        return('Educational Place')
    
    elif x in ['Hotel','Residence Hall','Residence Hall/Dormitory']:
        return('Hotel & Residence Hall')
    
    elif x in ['Warehouse','Non-Refrigerated Warehouse', 'Refrigerated Warehouse', 'Self-Storage Facility', 'Self-Storage Facility\n']:
        return('Warehouse')
    
    elif x in ['Hospital','Laboratory', 'Senior Care Community']:
        return('Medical Center')
    
    elif x in ['Low-Rise Multifamily']:
        return('Other')
    
    else:
        return(x)

##### Il faudrait vérifier avec un spécialiste la cohérance de ces catégories 

In [ ]:
NonRes['PrincipalUse']=NonRes['PrimaryPropertyType'].apply(Principal_Use)

In [ ]:
proportion=NonRes['PrincipalUse'].value_counts()

fig,ax=plt.subplots()
proportion.plot.pie(figsize=(10,10), autopct='%1.1f%%', colormap='Oranges', fontsize=25)
_=ax.set_ylabel('')

# Exploration 

## Analyse univariée

In [ ]:
NonRes.describe()

In [ ]:
def skew_comments(x):
    if x>0:
        return('étalée à droite ')
    elif x<0 :
        return('étalée à gauche')
    else:
        return('symétrique')

In [ ]:
df=pd.DataFrame()
df['colonne']=numeric_cols
df['skewness']=skew(NonRes[numeric_cols], nan_policy='omit')
df['interprétation']=df['skewness'].apply(skew_comments)
df

##### Les variables cibles sont les plus asymétriques : il faut les passer au log en 1er afin d'améliorer les résultats. 
##### Et comme on a suffisemment de colonnes, on n'aura pas besoin de le faire pour les autres colonnes 

In [ ]:
fig = plt.figure(figsize = (15, 20))

for i in range(len(numeric_cols)):
    graphique=plt.subplot(6,3,i+1)
    NonRes.boxplot(column=numeric_cols[i])
    graphique.set_xticks([])
    graphique.set_title(numeric_cols[i])

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize = (15, 15))

for i in range(len(numeric_cols)):
    graphique=plt.subplot(6,3,i+1)
    NonRes[numeric_cols[i]].hist(density=True,bins=20)
    graphique.set_title(numeric_cols[i])

fig.tight_layout()

In [ ]:
proportion=NonRes['PrimaryPropertyType'].value_counts()

fig,ax=plt.subplots()
proportion.plot.pie(figsize=(10,10), autopct='%1.1f%%', colormap='Oranges')
ax.set_ylabel('')
_=ax.set_title("Répartition BRUTE des differents types d'usage majeur des bâtiments",fontsize=25 )

##### Il était donc bien utile de créer les catégories plus vastes car ici, tres peu lisible !!!

In [ ]:
proportion=NonRes['PrincipalUse'].value_counts()

fig,ax=plt.subplots()
proportion.plot.pie(figsize=(10,10), autopct='%1.1f%%', colormap='Oranges')
ax.set_ylabel('')
_=ax.set_title("Répartition des differents types d'usage majeur des bâtiments",fontsize=25 )

In [ ]:
proportion=NonRes['Neighborhood'].value_counts()

fig,ax=plt.subplots()
proportion.plot.pie(figsize=(10,10), autopct='%1.1f%%', colormap='Blues')
ax.set_ylabel('')
_=ax.set_title("Répartition des bâtiments en fonction de leur quartier",fontsize=25 )

## Analyse bivariée

In [ ]:
_=sns.pairplot(NonRes[numeric_cols])

In [ ]:
corr_matrix=NonRes[numeric_cols].corr()
corr_matrix

##### GHGE et SiteUse très corrélées entre elles 
##### SiteUse assez corrélée à Buildings (et Total) ; GHGE aussi mais dans de moindre proportions
##### Pas de variable corrélées négativement 

In [ ]:
fig,ax=plt.subplots(figsize=(8,8))
sns.heatmap(corr_matrix, cmap='brg')
_=plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor" )


In [ ]:
def scatter_LR(X,Y,sample, df):
    xtrain, xtest, ytrain, ytest = train_test_split(df[[X]], df[[Y]], test_size=0.3, random_state=0)

    lr = LinearRegression()
    lr_baseline = lr.fit(xtrain[[X]], ytrain)
    baseline_pred = lr_baseline.predict(xtest[[X]])
  
    slope=round(lr_baseline.coef_[0][0],2)
    intercept=round(lr_baseline.intercept_[0],2)
    R2=round(r2_score(ytest, baseline_pred),3)

    if intercept>=0:
        sgn='+'
    else:
        sgn='-'

    fig= plt.figure(figsize=(10,6))
    ax=plt.axes()

    sampled_data=df.sample(n=sample, replace=False, random_state=0)
  
    ax.plot(sampled_data[X], sampled_data[Y], 'bo', markersize = 5)
    ax.plot(xtest[[X]], baseline_pred, color="slategrey", linewidth = 1)


    plt.xlabel(X, fontsize=20)
    plt.ylabel(Y, fontsize=20)

    ax.annotate('y={}x{}{}'.format(slope,sgn,abs(intercept)), xy=(200,280),xycoords='axes pixels', size=20)
    ax.annotate('R²={}'.format(R2), xy=(200,250),xycoords='axes pixels', size=20)

##### Observons les comprtements des PropertyGFA Total, Parking et Buildings entre elles :

In [ ]:
scatter_LR("PropertyGFABuilding(s)","PropertyGFATotal", 1100, NonRes)

In [ ]:
scatter_LR("PropertyGFAParking","PropertyGFATotal", 1100, NonRes)

In [ ]:
scatter_LR("PropertyGFAParking","PropertyGFABuilding(s)", 1100, NonRes)

###### Redondance entre Total & Buildings mais, d'après la matrice de corrélation, Buildings est un peu plus corrélée aux 2 variables cibles  => on supprime Total

In [ ]:
NonRes = NonRes.drop("PropertyGFATotal",axis = 1)

# Preprocessing 

## OneHot Encoding 

In [ ]:
one_hot_PrincipalUse = pd.get_dummies(NonRes['PrincipalUse'], prefix='PcpleUse')
NonRes = NonRes.join(one_hot_PrincipalUse)

In [ ]:
one_hot_Neighborhood = pd.get_dummies(NonRes['Neighborhood'], prefix='Neighborhood')
NonRes = NonRes.join(one_hot_Neighborhood)

In [ ]:
NonRes.describe()

## Normalisation 

In [ ]:
normal_cols=['YearBuilt', 'NumberofFloors','PropertyGFAParking',
       'PropertyGFABuilding(s)','ENERGYSTARScore' ]

In [ ]:
mms=MinMaxScaler()
NonRes[normal_cols]=mms.fit_transform(NonRes[normal_cols])

## Passage au log

In [ ]:
def Log(x):
    if x==0:
        return(-np.inf)
    else:
        return(math.log(x))

In [ ]:
NonRes['Log_EnergyUse']=NonRes['SiteEnergyUse(kBtu)'].apply(Log)
NonRes['Log_GHGE']=NonRes['GHGEmissions(MetricTonsCO2e)'].apply(Log)

In [ ]:
NonRes.drop(NonRes[NonRes['Log_EnergyUse']==-np.inf].index, inplace=True)
NonRes.drop(NonRes[NonRes['Log_GHGE']==-np.inf].index, inplace=True)

In [ ]:
NonRes.replace([np.inf, -np.inf], np.nan).dropna(subset=['Log_EnergyUse', 'Log_GHGE'], how="all", inplace=True)

### ---------------

In [ ]:
NonRes.to_csv('P4_data.csv',index=False) 